# calculating closest location

In [65]:
from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v['lat'],v['lon'],p['lat'],p['lon']))

In [66]:
ng = []
with open('/home/junlinux/Desktop/INF552/HW3/Nog_good_geoNtractid.csv', 'r') as rd:
    for line in rd:
        ng.append(line.strip('\n'))
ng = ng[1:]

In [67]:
ng_location = {}
loc_data = []
for n in ng:
    temp = n.split(',')
    step2 = [temp[0].strip('('), temp[1].strip(')'), temp[2]]
    step3 = {
        (float(step2[0]), float(step2[1])) : step2[2]
    }
    temp_data = {'lat':float(step2[0]), 'lon':float(step2[1])}
    #print(step3)
    ng_location[(float(step2[0]), float(step2[1]))] = step2[2]
    loc_data.append(temp_data)

In [102]:
crime = []
with open('/home/junlinux/Desktop/INF552/HW3/Crime_Data_From_2014_LA_cleaned.csv', 'r') as rd:
    for line in rd:
        crime.append(line.strip('\n'))

In [137]:
import pandas
crimep = pandas.read_csv('/home/junlinux/Desktop/INF552/HW3/Crime_Data_From_2014_LA_cleaned.csv', header=None, dtype=str)
#crime.head()

In [138]:
index = [
    'DR Number', 'Date Reported', 'Date Occurred', 'Time Occurred', 'Area ID', 'Area Name', 'Reporting District', 'Crime Code', 'Crime Code Description', 'MO Codes', 'Victim Age', 'Victim Sex', 'Victim Descent', 'Premise Code', 'Premise Description', 'Weapon Used Code', 'Weapon Description', 'Status Code', 'Status Description', 'Crime Code 1', 'Crime Code 2', 'Crime Code 3', 'Crime Code 4', 'Address', 'Cross Street', 'Location', 'tract_id', 'GeoLocation'
        ]
crimep = crimep[1:]

In [134]:
index

['DR Number',
 'Date Reported',
 'Date Occurred',
 'Time Occurred',
 'Area ID',
 'Area Name',
 'Reporting District',
 'Crime Code',
 'Crime Code Description',
 'MO Codes',
 'Victim Age',
 'Victim Sex',
 'Victim Descent',
 'Premise Code',
 'Premise Description',
 'Weapon Used Code',
 'Weapon Description',
 'Status Code',
 'Status Description',
 'Crime Code 1',
 'Crime Code 2',
 'Crime Code 3',
 'Crime Code 4',
 'Address',
 'Cross Street',
 'Location',
 'tract_id',
 'GeoLocation']

In [131]:
for idx, row in crimep[:11].iterrows():
    #print(row[25])
    point = tuple(map(float, row[25].strip('()').split(',')))
    #print(point)
    p = closest(loc_data, {'lat':point[0], 'lon':point[1]})
    #print(p)

{'lat': 34.0677546377, 'lon': -118.451161008}
{'lat': 34.0468660776, 'lon': -118.261684944}
{'lat': 34.0468660776, 'lon': -118.261684944}
{'lat': 34.0465369873, 'lon': -118.251617302}
{'lat': 34.0411108496, 'lon': -118.243213506}
{'lat': 34.0411108496, 'lon': -118.243213506}
{'lat': 34.0475500398, 'lon': -118.247726237}
{'lat': 34.064482389, 'lon': -118.239272516}
{'lat': 34.0475500398, 'lon': -118.247726237}
{'lat': 34.0469940542, 'lon': -118.24159781}
{'lat': 34.0367637343, 'lon': -118.265897928}


In [72]:
len(loc_data)

1003

In [139]:
import csv
with open('/home/junlinux/Desktop/INF552/HW3/crime_tract_geo_test.csv', 'a', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(index)    
    #for idx, row in crimep[:11].iterrows():
    for idx, row in crimep.iterrows():
        #print(row[25])
        point = tuple(map(float, row[25].strip('()').split(',')))
        #print(point)
        p = closest(loc_data, {'lat':point[0], 'lon':point[1]})
        #print(p)
        #print('>>', p)
        #print('>>', ng_location[(p['lat'], p['lon'])])
        #print(':::', len(t+[ng_location[(p['lat'], p['lon'])]]))
        #print([str(ng_location[(p['lat'], p['lon'])])]+[str((p['lat'], p['lon']))])
        #t1 = [str(ng_location[(p['lat'], p['lon'])])]+[str((p['lat'], p['lon']))]
        #spamwriter.writerow(temp_line+t1)
        spamwriter.writerow(list(row)+[ng_location[(p['lat'], p['lon'])]]+[(p['lat'], p['lon'])])

In [61]:
import shapefile
sf = shapefile.Reader("/home/junlinux/Desktop/INF552/HW3/cb_2014_06_tract_500k/cb_2014_06_tract_500k.shp")
shapes = sf.shapes()
rec = sf.shapeRecords()

In [62]:
ca = []
for x in rec:
    if x.record[1] == '037':
        temp ={}
        temp['state id'] = x.record[1]
        temp['tract id'] = x.record[4]
        temp['type'] = x.shape.__geo_interface__["type"]
        temp['parts'] = x.shape.parts
        temp['points'] = x.shape.points
        ca.append(temp)

In [63]:
la_tract = []
import csv
with open('/home/junlinux/Desktop/INF552/HW3/2014_NotGood_health_LA_mini.csv', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        la_tract.append(row)
#len(la_tract)
tract_ids = [int(x[3]) for x in la_tract[1:]]

la_tdic = {}
for l in la_tract[1:]:
    la_tdic[int(l[3])] = l

In [64]:
la_shape = []
for c in ca:
    if int(c['tract id']) in tract_ids:
        la_shape.append(c)

In [6]:
def geometry(type_v, coord):
    """
    type_v: type info as string
    coord:  poinst of vectors from shape file. Assume only polygon.
    """
    temp = {}
    temp['type'] = type_v
    temp["coordinates"] = [
        [list(x) for x in coord]        
    ]
    return temp
geometry(la_shape[0]['type'] , la_shape[0]['points'])

def properties(name_v, heat_v):
    """
    name_v: tract_id from cencus tract
    heat_v: range of values. Bad health, number of crime, and etc
    """
    temp = {}
    temp['name'] = name_v
    temp['heat'] = heat_v
    return temp
properties(la_shape[0]['tract id'], 14.5)

def feature_gen(id_v, name_v, heat_v, type_v, coord):
    """
    id_v: unique value to identify feature
    name_v: tract_id from cencus tract
    heat_v: range of values. Bad health, number of crime, and etc
    type_v: type info as string
    coord:  poinst of vectors from shape file. Assume only polygon    
    """
    temp = {}
    temp['type'] = "Feature"
    temp['id'] = id_v
    temp['properties'] = properties(name_v, heat_v)
    temp['geometry'] = geometry(type_v, coord)
    return temp
feature_gen(1, la_shape[0]['tract id'], 14.5, la_shape[0]['type'] , la_shape[0]['points'])

def properties_mn(name_v, heat_idx, heat_v):
    """
    name_v: tract_id from cencus tract
    heat_idx: list of heat_v names. list of string.
    heat_v: list of heat values
    """
    temp = {}
    temp['name'] = name_v
    
    if len(heat_idx) != len(heat_v):
        # length of heat_idx and heat_v has to be same
        raise
    for i in range(len(heat_idx)):
        temp[heat_idx[i]] = heat_v[i]
    return temp
#properties(la_shape[0]['tract id'], 14.5)

def feature_gen_mn(id_v, name_v, heat_idx, heat_v, type_v, coord):
    temp = {}
    temp['type'] = "Feature"
    temp['id'] = id_v
    temp['properties'] = properties_mn(name_v, heat_idx, heat_v)
    temp['geometry'] = geometry(type_v, coord)
    return temp
#feature_gen(1, la_shape[0]['tract id'], 14.5, la_shape[0]['type'] , la_shape[0]['points'])

Health data geojson

In [9]:
geoj = {}
geoj["type"] = "FeatureCollection"
geoj["features"] = []
# basic geo json dictionary. Each feature will be appended to features list

#geoj["features"].append(feature_gen(1, la_shape[0]['tract id'], 14.5, la_shape[0]['type'] , la_shape[0]['points']))
cnt = 1
for sh in la_shape:
    #print(float(la_tdic[int(sh['tract id'])][0]))
    t_data = float(la_tdic[int(sh['tract id'])][0])
    t_shape = feature_gen('{:04}'.format(cnt), sh['tract id'], float(la_tdic[int(sh['tract id'])][0]), sh['type'] , sh['points'])
    cnt += 1
    geoj["features"].append(t_shape)
#la_tdic
len(geoj["features"])

import json
with open('/home/junlinux/Desktop/INF552/HW3/gjson/test01.geojson', 'w') as wr:
    json.dump(geoj, wr)

crime data geojson - number of crime

In [140]:
crimes = []
import csv
with open('/home/junlinux/Desktop/INF552/HW3/crime_tract_geo_test.csv', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        crimes.append(row)

In [141]:
crime_n_intract = {}
ctemp = {}
for c in crimes[1:]:
    if int(c[26]) in crime_n_intract:
        crime_n_intract[int(c[26])].append(c)
        ctemp[int(c[26])] += 1
    else:
        crime_n_intract[int(c[26])] = [c]
        ctemp[int(c[26])] = 1

In [142]:
cn_geo = {}
cn_geo["type"] = "FeatureCollection"
cn_geo["features"] = []
# basic geo json dictionary. Each feature will be appended to features list

#geoj["features"].append(feature_gen(1, la_shape[0]['tract id'], 14.5, la_shape[0]['type'] , la_shape[0]['points']))
cnt = 1
for sh in la_shape:
    #print(float(la_tdic[int(sh['tract id'])][0]))
    t_data = ctemp[int(sh['tract id'])]
    t_shape = feature_gen('{:04}'.format(cnt), sh['tract id'], t_data, sh['type'] , sh['points'])
    cnt += 1
    cn_geo["features"].append(t_shape)
#la_tdic
len(cn_geo["features"])

import json
with open('/home/junlinux/Desktop/INF552/HW3/gjson/crime_N.geojson', 'w') as wr:
    json.dump(cn_geo, wr)

See the number of crimes in 100 range

In [143]:
import numpy as np
min_v = 3.0
max_v = 1526.0
step_v = 100.0
idx1 = list(np.arange(min_v, max_v, step_v))
#idx1 = [float(x) for x in range(5,32,5)]
import bisect
from collections import defaultdict
test_dic = defaultdict(list)
c_list = [(k, ctemp[k]) for k in ctemp.keys()]
c_list.sort(key=lambda a: a[1])

for x,y in c_list:
    #Now  find the index where the value from the list can fit in the 
    #b1 list, bisect uses binary search so this is an O(log n ) step.
    # use this returned index as key and append the list to that key.
    ind = bisect.bisect_right(idx1,float(y))
    test_dic[ind].append([x,y])
    
cnt = 0
for v in test_dic.values():
    if cnt<len(idx1)-1:
        print('{} ~ {}: '.format(idx1[cnt], idx1[cnt+1]), len(v))
        cnt += 1
    else:
        print('{} ~ max[{}]: '.format(idx1[cnt], max_v, len(v)))

3.0 ~ 103.0:  189
103.0 ~ 203.0:  469
203.0 ~ 303.0:  202
303.0 ~ 403.0:  96
403.0 ~ 503.0:  20
503.0 ~ 603.0:  11
603.0 ~ 703.0:  6
703.0 ~ 803.0:  4
803.0 ~ 903.0:  1
903.0 ~ 1003.0:  2
1003.0 ~ 1103.0:  1
1103.0 ~ 1203.0:  1
1203.0 ~ 1303.0:  1


# Time crime occurred

In [144]:
midnight = 1
morning  = 2
day      = 3
night    = 4
def time_check(time_v):
    if 0 <= time_v < 600:
        return midnight
    elif 600 <= time_v < 1200:
        return morning
    elif 1200 <= time_v < 1800:
        return day
    else:
        return night

In [145]:
crime_n_intract = {}
time_temp = {}
for c in crimes[1:]:
    if int(c[26]) in time_temp:
        t_v = time_check(int(c[3]))
        time_temp[int(c[26])][t_v].append(c)
        #ctemp[int(c[26])] += 1
    else:
        # tract_id, midnight, morning, day, night
        time_temp[int(c[26])] = [int(c[26]), [], [], [], []]
        t_v = time_check(int(c[3]))
        time_temp[int(c[26])][t_v].append(c)
        #ctemp[int(c[26])] = 1

In [146]:
time_list = []
for t in time_temp.values():
    #print('id: {} >> {}, {}, {}, {}'.format(t[0], len(t[1]), len(t[2]), len(t[3]), len(t[4])))
    time_list.append([t[0], len(t[1]), len(t[2]), len(t[3]), len(t[4])])
#print(len(time_temp))
#print(len(time_list))

geojson dump

In [147]:
time_idx = ['midnight', 'morning', 'day', 'night']
ct_geo = {}
ct_geo["type"] = "FeatureCollection"
ct_geo["features"] = []
# basic geo json dictionary. Each feature will be appended to features list

#geoj["features"].append(feature_gen(1, la_shape[0]['tract id'], 14.5, la_shape[0]['type'] , la_shape[0]['points']))
cnt = 1
for sh in la_shape:
    #print(float(la_tdic[int(sh['tract id'])][0]))
    t_data = time_temp[int(sh['tract id'])][1:] # get rid of tract id
    t_data = [len(x) for x in t_data]
    t_shape = feature_gen_mn('{:04}'.format(cnt), sh['tract id'], time_idx, t_data, sh['type'] , sh['points'])
    cnt += 1
    ct_geo["features"].append(t_shape)
#la_tdic
len(ct_geo["features"])

import json
with open('/home/junlinux/Desktop/INF552/HW3/gjson/time_crime_occurred.geojson', 'w') as wr:
    json.dump(ct_geo, wr)

# Crime code part

In [148]:
c_code = [int(x[7]) for x in crimes[1:]]
c_des = [x[8] for x in crimes[1:]]
c_cnd = [(int(x[7]), x[8]) for x in crimes[1:]]
cs = list(set(c_cnd))

In [149]:
codes = []
#word_list = ['theft', 'burglary', 'assault', 'robbery', 'shoot', 'shot', 'battery', 'stolen', 'child', 'homicide', 'drug', 'dumping', 'intimate', 'lynching', 'pandering', 'firearm', 'unlawful', 'vandalism']
word_list = ['theft', 'burglary', 'assault', 'robbery', 'shoot', 'shot']
for c in cs:
    if any(word in c[1].lower() for word in word_list):
        codes.append(c)

In [150]:
crime5_intract = {}
ctemp5 = {}
ctemp_other = {}
code_5 = [220,625,670,349,668,473,230,310,475,441,446,624,250,236,231,347,440,753,251,341,350,664,235,330,474,210,320,420,450,470,627,626,410,331,421]
code_other = [625,931,860,870,624,865,806,236,753,922,251,436,110,435,813,810,740,627,949,626]
for c in crimes[1:]:
    if any(code == int(c[7]) for code in code_5):
        if int(c[26]) in ctemp5:
            #crime_n_intract[int(c[26])].append(c)
            ctemp5[int(c[26])] += 1
        else:
            #crime_n_intract[int(c[26])] = [c]
            ctemp5[int(c[26])] = 1
    if any(code == int(c[7]) for code in code_other):
        if int(c[26]) in ctemp_other:
            #crime_n_intract[int(c[26])].append(c)
            ctemp_other[int(c[26])] += 1
        else:
            #crime_n_intract[int(c[26])] = [c]
            ctemp_other[int(c[26])] = 1

In [151]:
cs_list = [x for x in ctemp5.values()]
csother_list = [x for x in ctemp_other.values()]

import numpy as np
min_v = 0.0
max_v = 501.0
step_v = 17.0
idx1 = list(np.arange(min_v, max_v, step_v))
idx1 = idx1[:7]
idx1.append(max_v)

#idx1 = [float(x) for x in range(5,32,5)]
import bisect
from collections import defaultdict
test_dic = defaultdict(list)
c5_list = [[k, ctemp_other[k]] for k in ctemp_other.keys()]
c5_list.sort(key=lambda a: a[1])

for x,y in c5_list:
    #Now  find the index where the value from the list can fit in the 
    #b1 list, bisect uses binary search so this is an O(log n ) step.
    # use this returned index as key and append the list to that key.
    ind = bisect.bisect_right(idx1,float(y))
    test_dic[ind].append([x,y])
    
cnt = 0
for v in test_dic.values():
    if cnt<len(idx1)-1:
        print('{} ~ {}: '.format(idx1[cnt], idx1[cnt+1]), len(v))
        cnt += 1
    else:
        print('{} ~ max[{}]: '.format(idx1[cnt], max_v, len(v)))

0.0 ~ 17.0:  154
17.0 ~ 34.0:  287
34.0 ~ 51.0:  248
51.0 ~ 68.0:  141
68.0 ~ 85.0:  70
85.0 ~ 102.0:  31
102.0 ~ 501.0:  66
501.0 ~ max[501.0]: 


In [152]:
cn5_geo = {}
cn5_geo["type"] = "FeatureCollection"
cn5_geo["features"] = []
# basic geo json dictionary. Each feature will be appended to features list

#geoj["features"].append(feature_gen(1, la_shape[0]['tract id'], 14.5, la_shape[0]['type'] , la_shape[0]['points']))
cnt = 1
for sh in la_shape:
    #print(float(la_tdic[int(sh['tract id'])][0]))
    if int(sh['tract id']) in ctemp_other:
        t_data = ctemp_other[int(sh['tract id'])]
    else:
        t_data = 0
    t_shape = feature_gen('{:04}'.format(cnt), sh['tract id'], t_data, sh['type'] , sh['points'])
    cnt += 1
    cn5_geo["features"].append(t_shape)
#la_tdic
len(cn5_geo["features"])

import json
with open('/home/junlinux/Desktop/INF552/HW3/gjson/crime_other.geojson', 'w') as wr:
    json.dump(cn5_geo, wr)

# Final Data Generation

Less feature data

In [153]:
final_data = {}
for k in la_tdic.keys():
    final_data[k] = [int(la_tdic[k][3]), int(la_tdic[k][1]), float(la_tdic[k][0])]

In [154]:
for k in final_data.keys():
    final_data[k].append(ctemp[k])
    
for k in final_data.keys():
    if k in ctemp5:
        final_data[k].append(ctemp5[k])
    else:
        final_data[k].append(0)
    if k in ctemp_other:
        final_data[k].append(ctemp_other[k])
    else:
        final_data[k].append(0)    
        
for k in final_data.keys():
    final_data[k].extend( [len(x) for x in time_temp[k][1:]] )

In [155]:
def bad_hrange(h_v):
    if 5.0 <= h_v < 9.0:
        return 1
    elif 9.0 <= h_v < 13.0:
        return 2
    elif 13.0 <= h_v < 17.0:
        return 3
    elif 17.0 <= h_v < 21.0:
        return 4
    elif 21.0 <= h_v < 25.0:
        return 5
    elif 25.0 <= h_v < 29.0:
        return 6
    else:
        return 7
    
final_data_r = {}
for k in final_data.keys():
    temp = [final_data[k][0]]
    temp.append(final_data[k][1])
    #temp.append(pop_range(final_data[k][1]))
    
    temp.append(final_data[k][2])
    temp.append(bad_hrange(final_data[k][2]))
    
    temp.append(final_data[k][3])
    #temp.append(ncrime_range(final_data[k][3]))
    
    temp.append(final_data[k][4])
    #temp.append(n5crime_range(final_data[k][4]))
    
    temp.append(final_data[k][5])
    #temp.append(nothercrime_range(final_data[k][5]))
    
    temp.append(final_data[k][6])
    #temp.append(mid_range(final_data[k][6]))
    
    temp.append(final_data[k][7])
    #temp.append(morning_range(final_data[k][7]))
    
    temp.append(final_data[k][8])
    #temp.append(day_range(final_data[k][8]))
    
    temp.append(final_data[k][9])
    #temp.append(night_range(final_data[k][9]))
    final_data_r[k] = temp
       

In [156]:
import csv
idx_lb = ['tract_id', 'population', 'bad_health', 'bad_health_range', 'NofCrime', 'Nof5CategoryCrime', 'NofOtherCrime', 'midnightCrime', 'morningCrime', 'dayCrime', 'nightCrime']
with open('/home/junlinux/Desktop/INF552/HW3/merged_data(less_features)02.csv', 'a', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(idx_lb)
    #spamwriter.writerow(test[0])
    for k in final_data_r.keys():
        spamwriter.writerow(final_data_r[k])

More crime data

In [159]:
ta = dtype2={'DR Number':np.int64,
 'Date Reported':np.str_,
 'Date Occurred': np.str_,
 'Time Occurred':np.int64,
 'Area ID':np.int64,
 'Area Name': np.str_,
 'Reporting District':np.int64,
 'Crime Code':np.int64,
 'Crime Code Description': np.str_,
 'MO Codes': np.str_,
 'Victim Age':np.str_,
 'Victim Sex': np.str_,
 'Victim Descent': np.str_,
 'Premise Code':np.str_,
 'Premise Description': np.str_,
 'Weapon Used Code':np.str_,
 'Weapon Description':np.str_,
 'Status Code': np.str_,
 'Status Description': np.str_,
 'Crime Code 1':np.str_,
 'Crime Code 2':np.str_,
 'Crime Code 3':np.str_,
 'Crime Code 4':np.str_,
 'Address':np.str_,
 'Cross Street': np.str_,
 'Location ': np.str_,
 'tract_id':np.int64,
 'GeoLocation': np.str_}
len(ta)

28

In [161]:
import numpy as np
import pandas as pd
dtype2={'DR Number':np.int64,
 'Date Reported':np.str_,
 'Date Occurred': np.str_,
 'Time Occurred':np.int64,
 'Area ID':np.int64,
 'Area Name': np.str_,
 'Reporting District':np.int64,
 'Crime Code':np.int64,
 'Crime Code Description': np.str_,
 'MO Codes': np.str_,
 'Victim Age':np.str_,
 'Victim Sex': np.str_,
 'Victim Descent': np.str_,
 'Premise Code':np.str_,
 'Premise Description': np.str_,
 'Weapon Used Code':np.str_,
 'Weapon Description':np.str_,
 'Status Code': np.str_,
 'Status Description': np.str_,
 'Crime Code 1':np.str_,
 'Crime Code 2':np.str_,
 'Crime Code 3':np.str_,
 'Crime Code 4':np.str_,
 'Address':np.str_,
 'Cross Street': np.str_,
 'Location ': np.str_,
 'tract_id':np.int64,
 'GeoLocation': np.str_}

#data = pd.read_csv('/home/junlinux/Desktop/INF552/HW3/crime_tract_geo_test.csv',dtype=dtype2, usecols=dtype2)
data = pd.read_csv('/home/junlinux/Desktop/INF552/HW3/crime_tract_geo_test.csv',dtype=dtype2)

In [163]:
data.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location,tract_id,GeoLocation
0,140404019,01/01/2014,01/01/2013,400,4,Hollenbeck,429,510,VEHICLE - STOLEN,NaN,...,Invest Cont,510,NaN,NaN,NaN,3800 LIFUR AV,NaN,"(34.0887, -118.167)",6037201501,"(34.0865636765, -118.164655519)"
1,130801740,01/01/2014,01/01/2014,40,8,West LA,837,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",0416 0400 0429,...,Invest Cont,230,NaN,NaN,NaN,GLENROCK,OPHIR,"(34.0662, -118.4507)",6037265304,"(34.0677546377, -118.451161008)"
2,140104019,01/01/2014,01/01/2014,100,1,Central,182,624,BATTERY - SIMPLE ASSAULT,416,...,Invest Cont,624,NaN,NaN,NaN,900 S FIGUEROA ST,NaN,"(34.0462, -118.2628)",6037207710,"(34.0468660776, -118.261684944)"
3,140104020,01/01/2014,01/01/2014,100,1,Central,182,624,BATTERY - SIMPLE ASSAULT,416,...,Invest Cont,624,NaN,NaN,NaN,900 S FIGUEROA ST,NaN,"(34.0462, -118.2628)",6037207710,"(34.0468660776, -118.261684944)"
4,140104037,01/01/2014,01/01/2014,325,1,Central,174,946,OTHER MISCELLANEOUS CRIME,1212,...,Invest Cont,946,NaN,NaN,NaN,LOS ANGELES ST,8TH ST,"(34.0419, -118.2523)",6037207301,"(34.0465369873, -118.251617302)"


In [164]:
t1 = data['Victim Descent']
temp = set(t1)
base_desc = {}
for x in temp:
    base_desc[x] = 0

In [165]:
crime_n_intract = {}
c_desc = {}
for c in crimes[1:]:
    if int(c[26]) in crime_n_intract:
        crime_n_intract[int(c[26])].append(c)
        ctemp[int(c[26])] += 1
    else:
        #crime_n_intract[int(c[26])] = [c]
        ctemp[int(c[26])] = 1

In [166]:
crime_n_intract = {}
c_desc = {}
for index, row in data.iterrows():
    if row['tract_id'] in tract_ids:
        if row['tract_id'] in c_desc:
            c_desc[ row['tract_id'] ][ row['Victim Descent'] ] += 1
        else:
            c_desc[ row['tract_id'] ] = dict(base_desc)
            c_desc[ row['tract_id'] ][ row['Victim Descent'] ] += 1    
            
import csv
keys = [k for k in c_desc.keys()]
keys.sort()
idx_lb = [x for x in base_desc.keys()]
idx_lb = [idx_lb[0]]+sorted(idx_lb[1:])
#print(idx_lb)
idx_lb2 = ['v_'+str(x) for x in idx_lb]

with open('/home/junlinux/Desktop/INF552/HW3/Final_data/00_victim_desc.csv', 'a', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(idx_lb2+['tract_id2'])
    #spamwriter.writerow(mrd[0])
    for k in keys:
        line = []
        for kx in idx_lb:
            line.append(c_desc[k][kx])
        spamwriter.writerow(line+[k])            

In [167]:
base_status = {}
for x in set(data['Status Code']):
    base_status[x] = 0
    
c_status = {}
for index, row in data.iterrows():
    if row['tract_id'] in tract_ids:
        if row['tract_id'] in c_status:
            c_status[ row['tract_id'] ][ row['Status Code'] ] += 1
        else:
            c_status[ row['tract_id'] ] = dict(base_status)
            c_status[ row['tract_id'] ][ row['Status Code'] ] += 1    
            
idx_lb = [x for x in base_status.keys()]
idx_lb2 = ['status_code_'+str(x) for x in idx_lb]

import csv
keys = [k for k in c_status.keys()]
keys.sort()

with open('/home/junlinux/Desktop/INF552/HW3/Final_data/00_status.csv', 'a', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(idx_lb2+['tract_id2'])
    #spamwriter.writerow(mrd[0])
    for k in keys:
        line = []
        for kx in idx_lb:
            line.append(c_status[k][kx])
        spamwriter.writerow(line+[k])

In [168]:
base_weapon = {}
for x in set(data['Weapon Used Code']):
    base_weapon[x] = 0
    
c_weapon = {}
for index, row in data.iterrows():
    if row['tract_id'] in tract_ids:
        if row['tract_id'] in c_weapon:
            c_weapon[ row['tract_id'] ][ row['Weapon Used Code'] ] += 1
        else:
            c_weapon[ row['tract_id'] ] = dict(base_weapon)
            c_weapon[ row['tract_id'] ][ row['Weapon Used Code'] ] += 1  
            
idx_lb = [x for x in base_weapon.keys()]
idx_lb = [idx_lb[0]]+sorted(idx_lb[1:])
#print(idx_lb)
idx_lb2 = ['w_'+str(x) for x in idx_lb]
#print(idx_lb2)

keys = [k for k in c_weapon.keys()]
keys.sort()

with open('/home/junlinux/Desktop/INF552/HW3/Final_data/00_weapon.csv', 'a', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(idx_lb2+['tract_id2'])
    #spamwriter.writerow(mrd[0])
    for k in keys:
        line = []
        for kx in idx_lb:
            line.append(c_weapon[k][kx])
        spamwriter.writerow(line+[k])

In [169]:
base_premise = {}
for x in set(data['Premise Code']):
    base_premise[x] = 0
    
c_premise = {}
for index, row in data.iterrows():
    if row['tract_id'] in tract_ids:
        if row['tract_id'] in c_premise:
            c_premise[ row['tract_id'] ][ row['Premise Code'] ] += 1
        else:
            c_premise[ row['tract_id'] ] = dict(base_premise)
            c_premise[ row['tract_id'] ][ row['Premise Code'] ] += 1  
            
idx_lb = [x for x in base_premise.keys()]
#print(idx_lb)
idx_lb = [idx_lb[0]]+sorted(idx_lb[1:])
#print(idx_lb)
idx_lb2 = ['p_'+str(x) for x in idx_lb]

keys = [k for k in c_premise.keys()]
keys.sort()

with open('/home/junlinux/Desktop/INF552/HW3/Final_data/00_premise.csv', 'a', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(idx_lb2+['tract_id2'])
    #spamwriter.writerow(mrd[0])
    for k in keys:
        line = []
        for kx in idx_lb:
            line.append(c_premise[k][kx])
        spamwriter.writerow(line+[k])

In [171]:
k_temp = [x[:2] for x in set(data['Date Occurred'])]
k_temp = list(set(k_temp))
base_month = dict((x,0) for x in k_temp)

c_month = {}
for index, row in data.iterrows():
    if row['tract_id'] in tract_ids:
        if row['tract_id'] in c_month:
            c_month[ row['tract_id'] ][ row['Date Occurred'][:2] ] += 1
        else:
            c_month[ row['tract_id'] ] = dict(base_month)
            c_month[ row['tract_id'] ][ row['Date Occurred'][:2] ] += 1  
            
idx_lb = [x for x in base_month.keys()]
idx_lb.sort()
idx_lb2 = ['month_'+str(x) for x in idx_lb]
#print(idx_lb)

keys = [k for k in c_month.keys()]
keys.sort()

#for k in c_month.keys():
#    print(k, ": ", c_month[k])
    
with open('/home/junlinux/Desktop/INF552/HW3/Final_data/00_time_occurred.csv', 'a', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(idx_lb2+['tract_id2'])
    #spamwriter.writerow(mrd[0])
    for k in keys:
        line = []
        for kx in idx_lb:
            line.append(c_month[k][kx])
        spamwriter.writerow(line+[k])    

In [ ]:
idx_lb = [x for x in base_desc.keys()]
idx_lb = [x for x in base_status.keys()]

In [300]:
import pandas
crime_premise = pandas.read_csv('/home/junlinux/Desktop/INF552/HW3/Final_data/00_premise.csv', header=None, dtype=str)
crime_status = pandas.read_csv('/home/junlinux/Desktop/INF552/HW3/Final_data/00_status.csv', header=None, dtype=str)
crime_time = pandas.read_csv('/home/junlinux/Desktop/INF552/HW3/Final_data/00_time_occurred.csv', header=None, dtype=str)
crime_vdesc = pandas.read_csv('/home/junlinux/Desktop/INF552/HW3/Final_data/00_victim_desc.csv', header=None, dtype=str)
crime_weapon = pandas.read_csv('/home/junlinux/Desktop/INF552/HW3/Final_data/00_weapon.csv', header=None, dtype=str)
base_data = pd.read_csv("/home/junlinux/Desktop/INF552/HW3/merged_data(less_features)02.csv",header=None, dtype=str)

#crime.head()

In [301]:
base_data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,tract_id,population,bad_health,bad_health_range,NofCrime,Nof5CategoryCrime,NofOtherCrime,midnightCrime,morningCrime,dayCrime,nightCrime
1,6037101110,4731,14.1,3,148,65,45,19,29,46,54
2,6037101122,3664,10.4,2,39,10,10,2,9,11,17
3,6037101210,5990,17.1,4,204,105,59,34,33,59,78
4,6037101220,3363,15.3,3,160,68,48,25,33,50,52


In [302]:
base_data_idx = base_data[:1]
base_data = base_data[1:]
crime_vdesc_idx = crime_vdesc[:1]
crime_vdesc = crime_vdesc[1:]
crime_status_idx = crime_status[:1]
crime_status = crime_status[1:]
crime_time_idx = crime_time[:1]
crime_time = crime_time[1:]
crime_premise_idx = crime_premise[:1]
crime_premise = crime_premise[1:]
crime_weapon_idx = crime_weapon[:1]
crime_weapon = crime_weapon[1:]


In [303]:
tdf = {k: [] for k in range(1,len(base_data)+1)}

for idx, row in base_data.iterrows():
    #print(idx, row)
    tdf[idx].extend(list(row))
    
for idx, row in crime_vdesc.iterrows():
    #print(idx, row)
    tdf[idx].extend(list(row)[:-1])
    
for idx, row in crime_status.iterrows():
    #print(idx, row)
    tdf[idx].extend(list(row)[:-1])
    
for idx, row in crime_time.iterrows():
    #print(idx, row)
    tdf[idx].extend(list(row)[:-1])
    
for idx, row in crime_premise.iterrows():
    #print(idx, row)
    tdf[idx].extend(list(row)[:-1])
    
for idx, row in crime_weapon.iterrows():
    #print(idx, row)
    tdf[idx].extend(list(row)[:-1])

In [304]:
label = []

for idx, row in base_data_idx.iterrows():
    #print(idx, row)
    label.extend(list(row))
    
for idx, row in crime_vdesc_idx.iterrows():
    #print(idx, row)
    label.extend(list(row)[:-1])
    
for idx, row in crime_status_idx.iterrows():
    #print(idx, row)
    label.extend(list(row)[:-1])
    
for idx, row in crime_time_idx.iterrows():
    #print(idx, row)
    label.extend(list(row)[:-1])
    
for idx, row in crime_premise_idx.iterrows():
    #print(idx, row)
    label.extend(list(row)[:-1])
    
for idx, row in crime_weapon_idx.iterrows():
    #print(idx, row)
    label.extend(list(row)[:-1])

In [305]:
label

['tract_id',
 'population',
 'bad_health',
 'bad_health_range',
 'NofCrime',
 'Nof5CategoryCrime',
 'NofOtherCrime',
 'midnightCrime',
 'morningCrime',
 'dayCrime',
 'nightCrime',
 'v_nan',
 'v_A',
 'v_B',
 'v_C',
 'v_D',
 'v_F',
 'v_G',
 'v_H',
 'v_I',
 'v_J',
 'v_K',
 'v_L',
 'v_O',
 'v_P',
 'v_U',
 'v_V',
 'v_W',
 'v_X',
 'v_Z',
 'status_code_AO',
 'status_code_AA',
 'status_code_13',
 'status_code_JA',
 'status_code_CC',
 'status_code_IC',
 'status_code_JO',
 'month_01',
 'month_02',
 'month_03',
 'month_04',
 'month_05',
 'month_06',
 'month_07',
 'month_08',
 'month_09',
 'month_10',
 'month_11',
 'month_12',
 'p_nan',
 'p_101',
 'p_102',
 'p_103',
 'p_104',
 'p_105',
 'p_106',
 'p_107',
 'p_108',
 'p_109',
 'p_110',
 'p_111',
 'p_112',
 'p_113',
 'p_114',
 'p_115',
 'p_116',
 'p_117',
 'p_118',
 'p_119',
 'p_120',
 'p_121',
 'p_122',
 'p_123',
 'p_124',
 'p_125',
 'p_126',
 'p_127',
 'p_128',
 'p_129',
 'p_130',
 'p_131',
 'p_132',
 'p_135',
 'p_136',
 'p_138',
 'p_139',
 'p_140

In [309]:
with open('/home/junlinux/Desktop/INF552/HW3/Final_data/merged_all_data_test.csv', 'a', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)    
    spamwriter.writerow(label)
    #spamwriter.writerow(mrd[0])
    keys = [x for x in tdf.keys()]
    keys.sort()
    for k in keys:
        spamwriter.writerow(tdf[k])

# After generation, population, bad_health, tract_id, morningCrime, dayCrime are manually excluded before running algorithms